In [1]:
import os
os.environ["PYTHONUSERBASE"] = "/rodata/gi_discovery/scripts"
os.environ["PYTHONPATH"] = "/rodata/gi_discovery/scripts"

In [3]:
import cv2
import numpy as np
from itertools import chain
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import glob
from pathlib import Path
import math
import pandas as pd
import json
from natsort import natsorted


In [4]:
image_dir = 'fasting_full/test_data'    #validation_data  test_data
patient_image_paths = {}
patients = []
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.lower().endswith('.bmp'):
            patient_name = os.path.basename(root)
            patients.append(patient_name)
            if patient_name not in patient_image_paths:
                patient_image_paths[patient_name] = []
            patient_image_paths[patient_name].append(os.path.join(root, file))

patient_list = sorted(set(patients))
for patient in patient_list:
    image_count = len(patient_image_paths[patient])
    print(f'{patient}: {image_count} images')

HVCONTROLES1: 147 images
HVCONTROLES14: 147 images
HVCONTROLES3: 148 images
HVfinal1: 120 images
Hvfinal2: 118 images


In [4]:
#read all the files in a list, crop the axis and save the crop images together #
'''
images = []
cropped_images = []

for patient, paths in patient_image_paths.items():
    # print(f"Patient: {patient}, Image Paths: {paths}")
    for image_file in paths:
        image = cv2.imread(image_file)
        images.append(image)
        cropped_image_name = "fasting_full/train_cropped/" + Path(image_file).stem + "_cropped.png"
        
        cropped_image = image[15:930, 95:1900]
        cropped_images.append(cropped_image)
        cv2.imwrite(cropped_image_name, cropped_image)
    # print(f" image count for : {patient}: is  ",len(images))

len(images)
'''

In [6]:
#read images from image from cropped image folder
image_dir = 'fasting_full/test_cropped/'
image_paths = glob.glob(image_dir + '*.png')
sorted_image_paths = natsorted(image_paths)
images = []
for image_file in sorted_image_paths:
    image = cv2.imread(image_file)
    images.append(image)

len(images) 

494

## Functions

In [19]:

#constants
#y-direction 915 pixel = 33cm
#x-direction 1805 pixel = 120 sec 
#long propagation = 10cm
max_length = 915

#calculated variables
max_width = 7.15*(1805/120)    #107.55 pixel             
min_width = 3.22*(1805/120)     #48.43 pixel
mean_width = 4.3*(1805/120)     #64.68 pixel

#decided variables (all in pixcel)
max_area = 2000
long_propagation = (915/33)*10    #277 pixcel
delta =  (1805/120)*4  #merge bounding box if the distance between them is delta in X-direction, 
                        #5sec, 75pixcel
alpha = (915/33)*3  #merge bounding box if the distance between them is alpha in Y-direction, 
                    #3cm, 83pixcel 
angle_threshold = np.radians(5)  #to indentify phase III propagations. maximum allowed
                                #angle difference 5 degree
angle_collision = np.radians(14) #detect collllision if angle difference 1.5
overlap_threshold=0.3
desired_width = mean_width
not_straight = (1805/120)*4  #time difference between top and bottom part of the bounding-box


#selecting colors and creating masks
def create_mask(imc):
    hsv = cv2.cvtColor(imc, cv2.COLOR_BGR2HSV)
    lower_blue = np.array([100, 150, 0])
    upper_blue = np.array([140, 255, 255])

    red_light = np.array([165, 87, 111], np.uint8)
    red_dark = np.array([180, 255, 255], np.uint8)

    blue_light = np.array([99, 115, 150], np.uint8)
    blue_dark = np.array([120, 255, 255], np.uint8)

    yellow_light = np.array([55, 60, 200], np.uint8)
    yellow_dark = np.array([60, 255, 255], np.uint8)

    green_light = np.array([65, 50, 50], np.uint8)
    green_dark = np.array([77, 255, 255], np.uint8)

    # Range Specification
    kernel = np.ones((15, 15), "uint8")
    red = cv2.inRange(hsv, red_light, red_dark)
    blue = cv2.inRange(hsv, blue_light, blue_dark)
    yellow = cv2.inRange(hsv, yellow_light, yellow_dark)
    green = cv2.inRange(hsv, green_light, green_dark)

    red = cv2.dilate(red, kernel)
    res_red = cv2.bitwise_and(imc, imc, mask=red)

    blue = cv2.dilate(blue, kernel)
    res_blue = cv2.bitwise_and(imc, imc, mask=blue)

    yellow = cv2.dilate(yellow, kernel)
    res_yellow = cv2.bitwise_and(imc, imc, mask=yellow)

    green = cv2.dilate(green, kernel)
    res_green = cv2.bitwise_and(imc, imc, mask=green)

    mask = red + yellow + green
    return(mask)


def min_area_rectangle(ctrs):
    roatated_box = []
    for ctr in ctrs:
        area = cv2.contourArea(ctr)
        if area > max_area:
            rect = cv2.minAreaRect(ctr)
            box = cv2.boxPoints(rect)
            box = np.intp(box) #make it integer
            roatated_box.append(box)
    bounding_box= sorted(roatated_box, key=lambda x:x[0, 0])
    no_artifact_boxes = remove_artifacts(bounding_box)
    if phase_three(no_artifact_boxes): 
        return (no_artifact_boxes, True)
    else: 
        width_modified_boxes = width_modified_boundingbox(no_artifact_boxes)       
        merge_boxes = join_box(width_modified_boxes)
        filtered_boxes = remove_overlapping_boxes(merge_boxes, overlap_threshold)
        positive_tilted_boxes = remove_negative_tilted(filtered_boxes)
        long_propagation_boxes = long_propagation_box(positive_tilted_boxes)
        return(long_propagation_boxes, False)   
    
#identify Phase III
def phase_three(bounding_box):
    phase_three = False
    for i in range(len(bounding_box)):
        parallel_count = 0
        for j in range(i+1, len(bounding_box)): 
            if are_boxes_parallel(bounding_box[i], bounding_box[j], angle_threshold)and box_distance(bounding_box[j], bounding_box[j-1])< max_width:
                    parallel_count += 1
        if parallel_count >= 11:     
            phase_three = True
        else:
            continue
    if phase_three:
            return(True)
    else:
            return(False)
    
def are_boxes_parallel(box1, box2, angle_threshold):
    angle1 = get_box_angle(box1)
    angle2 = get_box_angle(box2)
    angle_diff = np.abs(angle1 - angle2)
    return (angle_diff < angle_threshold)
def get_box_angle(box):
    vec2 = box[2] - box[1]
    angle2 = np.arctan2(vec2[1], vec2[0])
    return (np.abs(angle2))                     
def box_distance(box1, box2):
    min_distance = float('inf')
    for point1 in box1:
        for point2 in box2:
            distance = np.linalg.norm(np.array(point1) - np.array(point2))
            min_distance = min(min_distance, distance)
    return min_distance

#remove bounding boxes having bigger width than length
def remove_artifacts(bounding_box):
    long_boxes = []
    allow_straight = 20
    for idx in range(len(bounding_box)):
        h,w,time = get_height_width(bounding_box[idx])        
        if w >= h:
            continue       
        else:
            long_boxes.append(bounding_box[idx])        
    return(long_boxes)  

def get_height_width(box):
    p1,p2,p3,p4 = box
    angle = get_box_angle(box)
    if np.abs(angle) < np.pi/4:  
            h = abs(p2[1] - p1[1])
            w = abs(p1[0] - p4[0])
            time = abs(p1[0]-p2[0])
    else:  
        h = abs(p4[1] - p1[1])
        w = abs(p1[0] - p2[0])
        time = abs(p1[0] - p4[0])
    return(h,w, time)

#modify the bounding box if width is out of range
def width_modified_boundingbox(boxes):
    width_modified_boxes = []
    for box in boxes:
        h, w, time =get_height_width(box)
        p1,p2,p3,p4 = box
        current_width = w    
        # Check if the current width is within the desired range
        if current_width > max_width:            
            theta = np.arctan2(p2[1] - p1[1], p2[0] - p1[0])
            new_width = desired_width
            dx = 0.5 * (current_width - new_width) * np.cos(theta)
            dy = 0.5 * (current_width - new_width) * np.sin(theta)
            new_p1 = (int(p1[0] + dx), int(p1[1] + dy))
            new_p2 = (int(p2[0] - dx), int(p2[1] - dy))
            new_p3 = (int(p3[0] - dx), int(p3[1] - dy))
            new_p4 = (int(p4[0] + dx), int(p4[1] + dy))
            adjusted_pts = np.array(np.intp([new_p1, new_p2, new_p3, new_p4]))
            angle = get_box_angle(adjusted_pts)
            if np.abs(angle) < np.pi/4:  # p2, p1 Side is parallel or almost parallel to the x-axis
                h = abs(new_p2[1] - new_p1[1])
                w = abs(new_p1[0] - new_p4[0])
                time = abs(new_p1[0]-new_p2[0])
            else:  # p2 and p1 Side is parallel or almost parallel to the y-axis
                h = abs(new_p4[1] - new_p1[1])
                w = abs(new_p1[0] - new_p2[0])
                time = abs(new_p1[0] - new_p4[0])
            if w >= h:
                width_modified_boxes.append(box)
            elif time < not_straight:
                width_modified_boxes.append(box)
            else:
                width_modified_boxes.append(adjusted_pts)
        else:
            width_modified_boxes.append(box)
    return(width_modified_boxes)

#join bounding boxes based on selected rules
def join_box(bounding_box):
    sorted_box= sorted(bounding_box, key=lambda x:x[0, 0])
    merged_boxes = []
    indices_of_merged_boxes = [] 
    for i in range(len(sorted_box)):
        for j in range(i+1, len(sorted_box)):            
            min_x1 = min(point[0] for point in sorted_box[i])
            max_x1 = max(point[0] for point in sorted_box[i])
            min_y1 = min(point[1] for point in sorted_box[i])
            max_y1 = max(point[1] for point in sorted_box[i])
            min_x2 = min(point[0] for point in sorted_box[j])
            max_x2 = max(point[0] for point in sorted_box[j])
            min_y2 = min(point[1] for point in sorted_box[j])
            max_y2 = max(point[1] for point in sorted_box[j])
            # Check for collision along the x-axis based on box location
            if min_y1>=min_y2:
                top_x = min_x2
                bottom_x = min_x1 
            else:
                top_x = min_x1
                bottom_x = min_x2
            if top_x> bottom_x:
                dist_x = delta + 1
            else:
                dist_x = (max_x1-min_x2) 
            dist_y = min(abs(max_y1 - min_y2), abs(max_y2 - min_y1))            
            x_collision = (dist_x <= delta and dist_x>=0)
            y_collision = (dist_y <= alpha)

            # Check if there is an overall collision
            if x_collision and y_collision and are_boxes_parallel(sorted_box[i], sorted_box[j],angle_collision):
                merged_box = merge_boxes1([sorted_box[i], sorted_box[j]])
                merged_box_np = np.array(merged_box)
                merged_boxes.append(merged_box_np)
                indices_of_merged_boxes.append([i, j])
    indices_1d = list(chain.from_iterable(indices_of_merged_boxes)) 
    indices_set = set(indices_1d)              
    for i in range(len(sorted_box)):
        if i not in indices_set:
            merged_boxes.append(sorted_box[i])
    return(merged_boxes)

def merge_boxes1(boxes):
    points = np.vstack(boxes)
    rect = cv2.minAreaRect(points)
    return np.intp(cv2.boxPoints(rect))

def remove_overlapping_boxes(boxes, overlap_threshold):    
    result_boxes = []
    while boxes:
        current_box = boxes.pop(0)
        overlapping_boxes = [current_box]
        i = 0
        while i < len(overlapping_boxes):
            j = 0
            while j < len(boxes):
                if box_overlap(overlapping_boxes[i], boxes[j], overlap_threshold):
                    overlapping_boxes.append(boxes.pop(j))
                else:
                    j += 1
            i += 1        
        # Merge the overlapping boxes and append to result
        result_boxes.append(merge_boxes1(overlapping_boxes))    
    return result_boxes

def box_overlap(box1, box2, overlap_threshold):
    rect1 = cv2.minAreaRect(box1)
    rect2 = cv2.minAreaRect(box2)
    intersection = cv2.rotatedRectangleIntersection(rect1, rect2)
    if intersection[0]:
        inter_area = cv2.contourArea(np.intp(intersection[1]))
        box1_area = cv2.contourArea(np.intp(cv2.boxPoints(rect1)))
        box2_area = cv2.contourArea(np.intp(cv2.boxPoints(rect2)))
        return inter_area / min(box1_area, box2_area) > overlap_threshold
    else:
        return False
    
def remove_negative_tilted(filtered_boxes):
    correct_boxes = []
    for box in filtered_boxes:
        p1,p2,p3,p4 = box
        angle = get_box_angle(box)
        if np.abs(angle) < np.pi/4:
            vec = p2 - p1 
            box_tilt = np.arctan2(vec[1],vec[0])
        else:
            vec = p4 - p1
            box_tilt = np.arctan2(vec[1], vec[0])

        if box_tilt > 0:
                correct_boxes.append(box)
        else:
            continue
    return(correct_boxes)

def long_propagation_box(filtered_boxes):
    long_boxes = []
    for idx in range(len(filtered_boxes)):
        h,w,time = get_height_width(filtered_boxes[idx])
        if w >= h:
            continue
        elif time < (1805/120)*2:
            continue
        elif h< long_propagation:
            continue
        else:
            long_boxes.append(filtered_boxes[idx])

    return long_boxes    
    
def convert_seconds(sec):
    seconds=round(sec)
    hours = seconds // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f"{hours}: {minutes}: {seconds}"

#check different types of length and width
def four_length_width(pts):   
    p1_x, p1_y = pts[0, 0]
    p2_x, p2_y = pts[1,0]
    p3_x, p3_y = pts[2,0]
    p4_x, p4_y = pts[3, 0]    
    p1_y = max(p1_y, 0)
    p1_x = max(p1_x, 0)
    p2_x = max(p2_x, 0)
    p3_x = max(p3_x, 0)
    p3_y = max(p3_y,0)
    p4_y = max(p4_y, 0)
    p4_x = max(p4_x, 0)
    #top-middle and bottom-middle points
    tm_x = int((p2_x+p1_x)/2)
    bm_x = int((p3_x+p4_x)/2)
    bm_y = int((p3_y+p4_y)/2)   

    length_bm_1 = abs(bm_y - p1_y) 
    width_bm_tm = abs(tm_x-bm_x)
    #length can not ne more than 33cm
    length_bm_1 = min(length_bm_1, max_length)
    #convert pixcel to cm
    length_bm_1 = length_bm_1*(33/915)
    #convert pixcel to sec
    width_bm_tm = width_bm_tm*(120/1805)
    top_time = p1_x*(120/1805)
    top_position = p1_y*(33/915)
    end_position = p4_y*(33/915)
    if top_position>33:
        top_position =33
    if end_position>33:
        end_position =33
    return(top_time,top_position, end_position, length_bm_1,width_bm_tm)


## Bounding boxes

In [5]:
df_auto_generated_propagations = pd.DataFrame(columns= ['patients','image','mask_count'])

df_propagations = pd.DataFrame(columns=['patients','image','start(time)','start(sec)',
                                        'start(location)cm','end(location)cm','length','width','velocity'])
data_to_append = []
propagation_to_append = []

for patient in patient_list:
    duration = 0
    for image in sorted_image_paths:
        if (Path(image).stem).split('-')[0].upper() == patient.upper():
            im = cv2.imread(image)
            imbox = "fasting_full/test_box/" + Path(image).stem + "_box.png"
            # immask  = "new/mask/" + Path(image).stem 
            image_name = Path(image).stem
            mask = create_mask(im)
            ctrs, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
            bounding_box = []
            # print(f"image name : {image_name}")
            bounding_box, phase_iii = min_area_rectangle(ctrs)
            
            if len(bounding_box)==0:
                pass
            else: 
                sorted_box = sorted(bounding_box, key=lambda x:x[0, 0])           
                for idx in range(len(sorted_box)):
                    pts = np.array([sorted_box[idx]], np.int32).reshape(-1,1,2)
                    mask_name = Path(image).stem + f"_{idx}"
                    p1,p2,p3,p4 = pts
                    top_time,top_position,end_position, length_bm_1,width_bm_tm = four_length_width(pts)
                    start_point = duration + top_time
                    start_time = convert_seconds(start_point)
                    if phase_iii:
                        pass
                    else:
                        if width_bm_tm != 0:
                            velocity = length_bm_1/width_bm_tm
                        else:
                            velocity = 0
                        propagation_to_append.append({
                            'patients': patient,
                            'image': mask_name,
                            'start(time)':start_time,
                            'start(sec)': start_point,
                            'start(location)cm': top_position,
                            'end(location)cm' :end_position,
                            'length': length_bm_1,
                            'width': width_bm_tm,
                            'velocity': velocity,})                    
                    cv2.drawContours(im, [sorted_box[idx]], 0, (0, 0, 255), 5) 
            duration += 120

            count = len(sorted_box)start(time)','start(location)
            data_to_append.append({'patients':   patient,'image': image_name, 'mask_count': count})
            cv2.imwrite(imbox, im)     

df_auto_generated_propagations = pd.concat([df_auto_generated_propagations, pd.DataFrame(data_to_append)], ignore_index=True)
df_auto_generated_propagations ['image']= df_auto_generated_propagations['image'].apply(lambda x : x.split('/')[1])
df_auto_generated_propagations.to_csv('fasting_full/test_box/propagation_count.csv', index=None)


df_propagations= pd.concat([df_propagations, pd.DataFrame(propagation_to_append)], ignore_index=True)
df_propagations.to_csv('fasting_full/test_box/propagations.csv', index=None)
   